### Advent of Code 2023

This notebook contains my solutions for the Advent of Code (https://adventofcode.com/2023) programming challenge.

#### Day 1: Trebuchet?!
https://adventofcode.com/2023/day/1

In [10]:
# --- Test Input 1 ---

input1 = """1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet"""

# --- Test Input 1 ---

input2 = """two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen"""

In [11]:
# --- Part 1 ---
sum(int(n[0]+n[-1]) for n in [[i for i in filter(str.isnumeric, line)] for line in input1.splitlines()])

142

In [12]:
# --- Part 2 ---
for i,n in enumerate("one two three four five six seven eight nine".split()):
    input2 = input2.replace(n, n[0]+str(i+1)+n[2:])
sum(int(n[0]+n[-1]) for n in [[i for i in filter(str.isnumeric, line)] for line in input2.splitlines()])

281

#### Day 2: Cube Conundrum
https://adventofcode.com/2023/day/2

In [13]:
# --- Test Input ---

input = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"""

In [14]:
# --- Part 1 ---
bag = {"red": 12 , "green": 13 , "blue": 14}
idSum = 0
for game in input.splitlines():
    ID, _, S = game.partition(':')
    possible = True
    for s in S.split(';'):
        for cs in s.split(','):
            n, c = cs.split()
            if int(n) > bag[c]:
                possible = False
    if possible:
        idSum += int(ID.split()[1])
idSum

8

In [15]:
# --- Part 2 ---

score = 0
for game in input.splitlines():
    bag = {"red": 0 , "green": 0 , "blue": 0}
    ID, _, S = game.partition(':')
    possible = True
    for s in S.split(';'):
        for cs in s.split(','):
            n, c = cs.split()
            if int(n) > bag[c]:
                bag[c] = int(n)
    p = 1
    for v in bag.values(): p *= v
    score += p
score

2286

#### Day 3: Gear Ratios
https://adventofcode.com/2023/day/3

In [16]:
# --- Test Input ---

input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

In [17]:
# --- Part 1 ---

import numpy
a = numpy.array([list(l) for l in input.split()])
b = numpy.zeros(a.shape, 'i')
for x in range(a.shape[0]):
    for y in range(a.shape[1]):
        if not a[x,y].isnumeric():
            if a[x,y] != '.':
                b[x-1:x+2,y-1:y+2] += 1
            a[x,y] = ' '
sall = sum(int(n) for l in a if len(s := ''.join(l).split()) for n in s if n.isnumeric())
a[(b > 0) & (a != ' ')] = 'X'
sonly = sum(int(n) for l in a if len(s := ''.join(l).split()) for n in s if n.isnumeric())
print(sall - sonly)

4361


In [18]:

# --- Part 2 ---

a = numpy.array([list(l) for l in input.split()])
b = numpy.zeros(a.shape, 'i')
ind = numpy.zeros(a.shape, 'i') - 1
indexNumber = 0
values = []
for x in range(a.shape[0]):
    insideNumber = False
    digits = []
    for y in range(a.shape[1]):
        if a[x,y].isnumeric():
            ind[x,y] = indexNumber
            insideNumber = True
            digits.append(a[x,y])
        else:
            if insideNumber:
                insideNumber = False
                indexNumber += 1
                values.append(int(''.join(digits)))
                digits = []
    if digits:
        values.append(int(''.join(digits)))
        indexNumber += 1

s = 0
for x in range(a.shape[0]):
    for y in range(a.shape[1]):
        if a[x,y] == '*':
            iNum = [i for i in set(ind[x-1:x+2,y-1:y+2].ravel()) if i >= 0]
            if len(iNum) == 2:
                s += values[iNum[0]] * values[iNum[1]]

print(s)

467835


#### Day 4: Scratchcards
https://adventofcode.com/2023/day/4

In [19]:
# --- Test Input ---

input = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

In [20]:
# --- Part 1 + 2 ---

lines = input.splitlines()
s = 0  # Part 1
cardCount = [1] * len(lines)  # Part 2
for i,line in enumerate(lines):
    n1, n2 = line.split(':')[-1].split('|')
    points = len(set(n1.split()).intersection(n2.split()))
    s += int(2**(points-1))  # Part 1
    for j in range(i+1,i+1+points): cardCount[j] += cardCount[i]  # Part 2
print('part 1:', s)
print('part 2:', sum(cardCount))

part 1: 13
part 2: 30


#### Day 5: If You Give A Seed A Fertilizer
https://adventofcode.com/2023/day/5

In [1]:
# --- Test Input ---

input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [3]:
blocks = input.split('\n\n')
seeds = list(map(int,blocks[0].split(':')[1].split()))
transforms = [[list(map(int, line.split())) for line in b.split(':\n')[-1].split('\n')] for b in blocks[1:]]

intervals = set(((i,1) for i in seeds))        # -- part 1 --
intervals = set(zip(seeds[::2], seeds[1::2]))  # -- part 2 --

for transform in transforms:
    shiftedIntervals = set()
    for t in transform:
        yShifted, y, ny = t  # y: transformed interval
        restIntervals = set()
        for i,(x,nx) in enumerate(intervals): # x: original interval
            edges = sorted([(x,'x'), (x+nx,'x'), (y,'y'), (y+ny,'y')])
            order = ''.join(s for _,s in edges)
            parts = [(e, edges[i+1][0]-e) for i,(e,_) in enumerate(edges[:-1])]
            task = { # overlap cases
                # case: what to do with the 3 parts (x:keep, +:transform)
                "xxyy": "x  ", # non-overlapping
                "yyxx": "  x", # non-overlapping
                "xyxy": "x+ ", # x left overlap
                "yxyx": " +x", # x right overlap
                "xyyx": "x+x", # y inside x
                "yxxy": " + ", # x inside y
            }[order]
            for transType, (newX, newN) in zip(task, parts):
                if newN > 0:
                    if transType == 'x': restIntervals.add((newX, newN))
                    if transType == '+': shiftedIntervals.add((newX+yShifted-y, newN))
        intervals = restIntervals
    intervals = shiftedIntervals.union(restIntervals)
print(min(intervals)[0])

46


#### Day 6: Wait For It
https://adventofcode.com/2023/day/6

In [23]:
# --- Test Input ---

input = """Time:      7  15   30
Distance:  9  40  200"""

In [24]:
inp = input                  # --- part 1 ---
inp = input.replace(' ','')  # --- part 2 ---

import numpy
times = numpy.array(list(map(int, inp.splitlines()[0].split(':')[1].split())),'d')
distances = numpy.array(list(map(int, inp.splitlines()[1].split(':')[1].split())),'d')
tMin = numpy.ceil(times/2 - numpy.sqrt(times**2/4-distances) + 1E-12)
tMax = numpy.floor(times/2 + numpy.sqrt(times**2/4-distances) - 1E-12)
int(numpy.prod(tMax - tMin + 1))


71503

#### Day 7:
https://adventofcode.com/2023/day/7

In [25]:
# --- Test Input ---

input = """"""

#### Day 8:
https://adventofcode.com/2023/day/8

In [26]:
# --- Test Input ---

input = """"""

#### Day 9:
https://adventofcode.com/2023/day/9

In [27]:
# --- Test Input ---

testinput = """"""

#### Day 10:
https://adventofcode.com/2023/day/10

In [28]:
# --- Test Input ---

input = """"""


#### Day 11:
https://adventofcode.com/2023/day/11

In [29]:
# --- Test Input ---

input = """"""

#### Day 12:
https://adventofcode.com/2023/day/12

In [30]:
input = """"""

#### Day 13:
https://adventofcode.com/2023/day/13

In [31]:
# --- Test Input ---

input = """"""

#### Day 14:
https://adventofcode.com/2023/day/14

In [32]:
# --- Test Input ---

input = """"""

#### Day 15:
https://adventofcode.com/2023/day/15

#### Day 16

#### Day 17

#### Day 18

#### Day 19

#### Day 20

#### Day 21

#### Day 22

#### Day 23

#### Day 24